In [6]:
import pandas as pd
import tensorflow.keras as k
from sklearn.preprocessing import StandardScaler
import os

In [24]:
def extract_cluster_from_model(model_path, np_matrix, pd_dataframe, evaluate_model=False, save_to_csv=False, save_name='clustered', save_path='./data/'):
    model = k.models.load_model(model_path)
    
    if evaluate_model:
        model.evaluate(X, X)

    encoder = k.models.Model(inputs=model.input, outputs=model.get_layer("encoded").output)
    
    predictions = pd.DataFrame(encoder.predict(np_matrix), columns=['axis1', 'axis2'])
    predictions_with_info = pd.concat(
        [pd_dataframe[['dependency', 'calendar_year',]], predictions],
        axis=1
    )

    if save_to_csv: 
        predictions_with_info.to_csv(os.path.join(save_path, save_name+'.csv'), index=False)

    return predictions_with_info

In [ ]:
adjective_data = pd.read_csv('./data/ngram_dependencies_adjectives_wide.csv')  # if model estimated with normalized, adopt!
print(adjective_data.columns[:10])

X = adjective_data.iloc[:, 2:]
X = StandardScaler().fit_transform(X)

# Single Model

In [27]:
extract_cluster_from_model(
    r'./data/model_autoencoder.hdf5',
    np_matrix=X,
    pd_dataframe=adjective_data,
    evaluate_model=False,
    save_to_csv=True,
    save_path='./data',
    save_name='clustered'
)

4144/4144 [==============================] - 13s 3ms/step


,dependency,calendar_year,axis1,axis2
0,abatable,1910,13.478148,4.370971
1,abatable,1913,13.478159,4.371004
2,abatable,1917,13.478145,4.370956
3,abatable,1918,13.478145,4.370956
4,abatable,1930,13.478168,4.371020
...,...,...,...,...
132592,young,2015,4.739575,2.527182
132593,young,2016,4.739575,2.527182
132594,young,2017,4.739575,2.527182
132595,young,2018,4.739575,2.527182


# Model Progress

In [1]:
def calculate_change(np_matrix, pd_dataframe, source_path, source_name_pattern, target_path):
    files_in_path = os.listdir(source_path)
    model_checkpoints = [f for f in files_in_path if source_name_pattern in f]

    for model_checkpoint in model_checkpoints:
        extract_cluster_from_model(
            os.path.join(source_path, model_checkpoint),
            np_matrix=np_matrix,
            pd_dataframe=pd_dataframe,
            save_to_csv=True,
            save_path = target_path,
            save_name=model_checkpoint+'.csv'
        )

In [2]:
calculate_change(X, adjective_data, r'data\model_checkpoints\Step 1', 'Epoch', r'data\model_checkpoints\Step 1\as_data')

NameError: name 'X' is not defined